In [54]:
%pip install langchain_openai langchain langchain_experimental ollama

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [55]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import tool

In [56]:
import pandas as pd
from qdrant_client import QdrantClient,models
from sentence_transformers import SentenceTransformer


In [57]:
client = QdrantClient(url="http://localhost:6333")
collection_name = "no_coordinates_collection"
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


/home/davis/miniconda3/envs/AI/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [58]:
# embeddings = model.encode('900 Embarcadero Del Mar Isla Vista CA 93117')
# print(len(embeddings))
# output = client.search(
#         collection_name=f"{collection_name}",
#         query_filter=models.Filter(
#             must=[
#                 models.FieldCondition(
#                     key="categories",
#                     match=models.MatchValue(
#                         value='Restaurants',
#                     ),
#                 )
#             ]
#         ),
#         query_vector=embeddings,
#         limit=3,
#     )
# print(output)

# Define your tools!

For this we create a Langchain tool for retrieving weather information and use the `convert_to_openai_function` to create the function definition that we will feed into the model

In [59]:
class SearchInput(BaseModel):
    location: str = Field(description="Current location of user.")
    Category: str = Field(description="Type of business that the user wants to go.")
@tool("get_catagory", args_schema=SearchInput)
def get_category(location: str, Category: str) -> str:
    """Get a business recommendation given location and catagory of user's choice.\
        e.g. "Name of restaurant": Helena Avenue Bakery, "Location": 131 Anacapa St, Ste C Santa Barbara 93101 ,"Category": Food, Restaurants, Salad, Coffee & Tea """
    address_embedding = model.encode(location)
    # print(address_embedding)
    output = client.search(
        collection_name=f"{collection_name}",
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="categories",
                    match=models.MatchValue(
                        value=Category,
                    ),
                )
            ]
        ),
        search_params=models.SearchParams(hnsw_ef=128, exact=False),
        query_vector=address_embedding,
        limit=3,
    )
    print(output)
    return {"Name of restaurant": Name ,"Location": location, "Category": 73.4}

tools = [get_category]
functions = [convert_to_openai_function(t) for t in tools]
functions

[{'name': 'get_catagory',
  'description': 'get_catagory(location: str, Catagory: str) -> str - Get a business recommendation given location and catagory of user\'s choice.        e.g. "Name of restaurant": Helena Avenue Bakery, "Location": 131 Anacapa St, Ste C Santa Barbara 93101 ,"Catagory": Food, Restaurants, Salad, Coffee & Tea',
  'parameters': {'type': 'object',
   'properties': {'location': {'description': 'Current location of user.',
     'type': 'string'},
    'Catagory': {'description': 'Type of business that the user wants to go.',
     'type': 'string'}},
   'required': ['location', 'Catagory']}}]

# Invoke the Model

Now we can prompt the model and pass the functions 🥳

In [60]:
import ollama
import json

SYSTEM_PROMPT = f"""
You are an navigation assistant with access to these functions -
{json.dumps(functions, indent=4)}
"""

print (SYSTEM_PROMPT)


You are an navigation assistant with access to these functions -
[
    {
        "name": "get_catagory",
        "description": "get_catagory(location: str, Catagory: str) -> str - Get a business recommendation given location and catagory of user's choice.        e.g. \"Name of restaurant\": Helena Avenue Bakery, \"Location\": 131 Anacapa St, Ste C Santa Barbara 93101 ,\"Catagory\": Food, Restaurants, Salad, Coffee & Tea",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "description": "Current location of user.",
                    "type": "string"
                },
                "Catagory": {
                    "description": "Type of business that the user wants to go.",
                    "type": "string"
                }
            },
            "required": [
                "location",
                "Catagory"
            ]
        }
    }
]



In [61]:
import json
import re
from typing import Optional, Dict

def parse_function_call(input_str: str) -> Optional[Dict[str, any]]:
    """
    Parses a text string to find and extract a function call.
    The function call is expected to be in the format:
    <functioncall> {"name": "<function_name>", "arguments": "<arguments_json_string>"}

    Args:
        input_str (str): The text containing the function call.

    Returns:
        Optional[Dict[str, any]]: A dictionary with 'name' and 'arguments' if a function call is found,
                                  otherwise None.
    """
    # Regex pattern to extract 'name' and 'arguments'
    pattern = r'"name":\s*"([^"]+)",\s*"arguments":\s*\'(.*?)\''

    # Search with regex
    match = re.search(pattern, input_str)
    print(match)
    if match:
        try:
            name = match.group(1)
            arguments_str = match.group(2)
            arguments = json.loads(arguments_str)
            return {"name": name, "arguments": arguments}
        except json.JSONDecodeError:
            return None
    return None


In [62]:
messages = [
     {'role': 'system','content': SYSTEM_PROMPT}, 
     {'role': 'user','content': 'Im at 900 Embarcadero Del Mar, whats the best restaurant nearby?'},
]

response = ollama.chat(model='calebfahlgren/natural-functions', messages=messages)
message = (response['message']['content'])

In [63]:
message

'<functioncall> {"name": "get_catagory", "arguments": \'{"location": "900 Embarcadero Del Mar", "Catagory": "Food, Restaurants"}\'}'

# Call the Function

Here we pull the function out of `kwargs` and call our tool with the arguments

In [64]:
messages.append({'role': 'assistant', 'content': message}) # add ai response to history

function_call = parse_function_call(message) # parse out function call name and args into json
print(function_call)
if function_call and function_call.get("name") == "get_category":
    args = function_call.get("arguments")
    restaurant = get_category.run(args)

restaurant

<re.Match object; span=(16, 127), match='"name": "get_catagory", "arguments": \'{"location>
{'name': 'get_catagory', 'arguments': {'location': '900 Embarcadero Del Mar', 'Catagory': 'Food, Restaurants'}}


NameError: name 'restaurant' is not defined

In [ ]:
messages.append({'role': 'user', 'content': 'Function Response: ' + str(current_weather)})
messages

[{'role': 'system',
  'content': '\nYou are a weather assistant with access to these functions -\n[\n    {\n        "name": "get_weather_tool",\n        "description": "get_weather_tool(location: str) -> str - Get the current weather in a given location",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "location": {\n                    "description": "The city and state, e.g. San Francisco, CA",\n                    "type": "string"\n                }\n            },\n            "required": [\n                "location"\n            ]\n        }\n    }\n]\n'},
 {'role': 'user', 'content': 'What is the weather like in the Dallas, TX?'},
 {'role': 'assistant',
  'content': '<functioncall> {"name": "get_weather_tool", "arguments": \'{"location": "Dallas, TX"}\'}'},
 {'role': 'user',
  'content': "Function Response: {'location': 'Dallas, TX', 'fahrenheit': 73.4}"}]

In [ ]:
response = ollama.chat(model='natural-functions', messages=messages)
response['message']['content']

'The current weather in Dallas, TX is sunny and the temperature is around 73.4 degrees Fahrenheit'